In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec

import cmocean
import cmocean.cm as cmo
import seaborn as sns


import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
df_doc = pd.read_csv('../Dissertation_projects/Data/TOC/TOC_GOM.csv', sep=',', index_col=0)

#make all BDL a 0 
df_doc.loc[df_doc['NPOC_uM'] == 'BLD', 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] == 'BLD', 'TN_uM'] = 0

df_doc.loc[df_doc['NPOC_uM'] < 0, 'NPOC_uM'] = 0
df_doc.loc[df_doc['TN_uM'] < 0, 'TN_uM'] = 0

#fix date time 
df_doc['date_time'] = pd.to_datetime(df_doc['Date'])
df_doc['Date'] = df_doc['date_time'].dt.date

df_doc['NPOC_uM'] = pd.to_numeric(df_doc['NPOC_uM'], errors='coerce')
df_doc['NPOC_uM'] = df_doc['NPOC_uM'].where(pd.notna(df_doc['NPOC_uM']), other=np.nan)

df_doc['TN_uM'] = pd.to_numeric(df_doc['TN_uM'], errors='coerce')
df_doc['TN_uM'] = df_doc['TN_uM'].where(pd.notna(df_doc['TN_uM']), other=np.nan)

df_doc['Time'] = pd.to_numeric(df_doc['Time'], errors='coerce')
df_doc['Time'] = df_doc['Time'].where(pd.notna(df_doc['Time']), other=np.nan)

df_doc['code'] = df_doc['Location'] + '_' + df_doc['Campaign']

In [3]:
df_meta = pd.read_csv('../Dissertation_projects/Data/meta_data.csv', sep=',', index_col=0)

#fix date time 
df_meta['date_time'] = pd.to_datetime(df_meta['Date'])
df_meta['Date'] = df_meta['date_time'].dt.date

df_meta['code'] = df_meta['Station'] + '_' + df_meta['Trip']

In [4]:
df_doc_wc = df_doc[df_doc['Type'] == 'WC']

df_doc_wc

,Treatment,Campaign,Date,Location,NPOC Conc (mg/L),TN Conc (mg/L),Depth,Notes,Type,Time,NPOC_uM,TN_uM,dil_fac,date_time,code
Sample ID,,,,,,,,,,,,,,,
211901,P2F,GOM2021,2021-07-15,St.MK,1.22200,0.1955,16.7,NaN,WC,NaN,101.833333,13.964286,NaN,2021-07-15,St.MK_GOM2021
211902,P2F,GOM2021,2021-07-15,St.MK,1.22350,0.182,16.7,NaN,WC,NaN,101.958333,13.000000,NaN,2021-07-15,St.MK_GOM2021
211903,P2F,GOM2021,2021-07-15,St.MK,1.35050,0.11485,14.0,NaN,WC,NaN,112.541667,8.203571,NaN,2021-07-15,St.MK_GOM2021
211904,P2F,GOM2021,2021-07-15,St.MK,1.57650,BDL,14.0,NaN,WC,NaN,131.375000,0.323571,NaN,2021-07-15,St.MK_GOM2021
211905,P2F,GOM2021,2021-07-15,St.MK,1.45250,0.16770000000000002,10.0,NaN,WC,NaN,121.041667,11.978571,NaN,2021-07-15,St.MK_GOM2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225036,P2F,GOM22SU,2022-07-21,St.12,0.97345,0.066635,70.6,TN STD LOW,WC,NaN,81.120833,4.759643,NaN,2022-07-21,St.12_GOM22SU
225037,P2F,GOM22SU,2022-07-21,St.12,1.04600,0.076015,45.0,TN STD LOW,WC,NaN,87.166667,5.429643,NaN,2022-07-21,St.12_GOM22SU
225038,P2F,GOM22SU,2022-07-21,St.12,1.06300,0.07098,21.0,TN STD LOW,WC,NaN,88.583333,5.070000,NaN,2022-07-21,St.12_GOM22SU


In [5]:
#DOC slopes pore water 
df_doc_wc = df_doc_wc.dropna(subset=["NPOC_uM"])
df_doc_wc = df_doc_wc[df_doc_wc['NPOC_uM'] >= 1]

codes = df_doc_wc['code'].unique()

print(codes)
len(codes)

df_doc_wc

['St.MK_GOM2021' 'St.5B_GOM2021' 'St.4_GOM2021' 'St.14_GOM2021'
 'St.9_GOM2021' 'St.13_GOM2021' 'St.2_GOM2021' 'St.15_GOM2021'
 'St.16_GOM2021' 'St.11_GOM2021' 'St.7_GOM2021' 'St.ML_GOM2021'
 'St.5B_GOM21FA' 'St.6_GOM21FA' 'St.4_GOM21FA' 'St.MK_GOM21FA'
 'St.MK-2_GOM21FA' 'St.7_GOM21FA' 'St.14_GOM21FA' 'St.9_GOM21FA'
 'St.14.5_GOM21FA' 'St.15_GOM21FA' 'St.2_GOM21FA' 'St.16_GOM21FA'
 'St.12_GOM21FA' 'St.11_GOM21FA' 'St.13 _GOM21FA' 'St.16-2_GOM21FA'
 'St.7_GOM22SP' 'St.16_GOM22SP' 'St.5B_GOM22SP' 'St.15_GOM22SP'
 'St.14_GOM22SP' 'St.2_GOM22SP' 'St.13_GOM22SP' 'St.MK_GOM22SP'
 'St.4_GOM22SP' 'St.4_GOM22SU' 'St.5B_GOM22SU' 'St.13_GOM22SU'
 'St.2_GOM22SU' 'St.9_GOM22SU' 'St.9_GOM22SP' 'St.14_GOM22SU'
 'St.12_GOM22SU']


,Treatment,Campaign,Date,Location,NPOC Conc (mg/L),TN Conc (mg/L),Depth,Notes,Type,Time,NPOC_uM,TN_uM,dil_fac,date_time,code
Sample ID,,,,,,,,,,,,,,,
211901,P2F,GOM2021,2021-07-15,St.MK,1.22200,0.1955,16.7,NaN,WC,NaN,101.833333,13.964286,NaN,2021-07-15,St.MK_GOM2021
211902,P2F,GOM2021,2021-07-15,St.MK,1.22350,0.182,16.7,NaN,WC,NaN,101.958333,13.000000,NaN,2021-07-15,St.MK_GOM2021
211903,P2F,GOM2021,2021-07-15,St.MK,1.35050,0.11485,14.0,NaN,WC,NaN,112.541667,8.203571,NaN,2021-07-15,St.MK_GOM2021
211904,P2F,GOM2021,2021-07-15,St.MK,1.57650,BDL,14.0,NaN,WC,NaN,131.375000,0.323571,NaN,2021-07-15,St.MK_GOM2021
211905,P2F,GOM2021,2021-07-15,St.MK,1.45250,0.16770000000000002,10.0,NaN,WC,NaN,121.041667,11.978571,NaN,2021-07-15,St.MK_GOM2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225036,P2F,GOM22SU,2022-07-21,St.12,0.97345,0.066635,70.6,TN STD LOW,WC,NaN,81.120833,4.759643,NaN,2022-07-21,St.12_GOM22SU
225037,P2F,GOM22SU,2022-07-21,St.12,1.04600,0.076015,45.0,TN STD LOW,WC,NaN,87.166667,5.429643,NaN,2022-07-21,St.12_GOM22SU
225038,P2F,GOM22SU,2022-07-21,St.12,1.06300,0.07098,21.0,TN STD LOW,WC,NaN,88.583333,5.070000,NaN,2022-07-21,St.12_GOM22SU


In [6]:
mean_values = df_doc_wc.groupby('code')['NPOC_uM'].mean()
max_values = df_doc_wc.groupby('code')['Depth'].max()

result_df = pd.concat([mean_values, max_values], axis=1, keys=['mean_NPOC_uM', 'max_depth'])

result_df.index.name = 'code'

result_df['NPOC_uM_per_cubic_meter'] = result_df['mean_NPOC_uM'] * 1000
result_df['NPOC_uMol_Stock_wc'] = result_df['NPOC_uM_per_cubic_meter'] * result_df['max_depth']

result_df.to_csv('../Dissertation_projects/Data/Stats_data_sets/Water_col_DOC_stocks.csv')

result_df

,mean_NPOC_uM,max_depth,NPOC_uM_per_cubic_meter,NPOC_uMol_Stock_wc
code,,,,
St.11_GOM2021,112.170833,946.0,112170.833333,1.061136e+08
St.11_GOM21FA,59.130093,915.0,59130.092593,5.410403e+07
St.12_GOM21FA,56.828333,950.0,56828.333333,5.398692e+07
St.12_GOM22SU,94.259167,996.0,94259.166667,9.388213e+07
St.13 _GOM21FA,58.486106,448.0,58486.105535,2.620178e+07
St.13_GOM2021,72.902500,322.0,72902.500000,2.347460e+07
St.13_GOM22SP,78.753646,555.0,78753.645833,4.370827e+07
St.13_GOM22SU,15.029415,488.0,15029.414996,7.334355e+06
St.14.5_GOM21FA,96.443750,103.0,96443.750000,9.933706e+06
